# Ungraded Lab: Multiple LSTMs

## Download and Prepare the Dataset

In [1]:
import tensorflow_datasets as TFDS

# Download the subword encoded pretokenized dataset
dataset, info = TFDS.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

# Get the tokenizer
tokenizer = info.features['text'].encoder

2024-03-06 15:55:35.604000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 15:55:35.604122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 15:55:35.605786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 15:55:37.516878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-06 15:55:39.541231: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 256

# Get the train and test splits
train_data, test_data = dataset['train'], dataset['test']

# Shuffle the training data
train_dataset = train_data.shuffle(BUFFER_SIZE)

# Batch and pad the datasets to the maximum leangh of the sequences
train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_data.padded_batch(BATCH_SIZE)

# Build and Compile the Model

In [6]:
import numpy as np 

import tensorflow as tf
from  tensorflow.keras.layers import LSTM , Embedding, Dense, Bidirectional
from tensorflow.keras import Sequential

# Hyperparameters
batch_size = 1
timesteps = 20
features = 16
lstm_dim = 8

print(f'batch_size: {batch_size}')
print(f'timesteps (sequence length): {timesteps}')
print(f'features (embedding size): {features}')
print(f'lstm output units: {lstm_dim}')

# Define array input with random values
random_input = np.random.rand(batch_size,timesteps,features)
print(f'shape of input array: {random_input.shape}')

# Define LSTM that returns a single output
lstm_rs = LSTM(lstm_dim, return_sequences=True)
result = lstm_rs(random_input)
print(f"shape of lstm output(return_sequences=True): {result.shape}")

batch_size: 1
timesteps (sequence length): 20
features (embedding size): 16
lstm output units: 8
shape of input array: (1, 20, 16)
shape of lstm output(return_sequences=True): (1, 20, 8)


![LSTM Neurlas](/home/alrashidi/Downloads/LSTM.jpeg)

In [7]:
# Hyperparameters
embedding_dim = 64
lstm1_dim = 64
lstm2_dim = 32
dense_dim = 64

# Build the model 
model = Sequential([
    Embedding(tokenizer.vocab_size, embedding_dim),
    Bidirectional(LSTM(lstm1_dim, return_sequences=True)),
    Bidirectional(LSTM(lstm2_dim)),
    Dense(dense_dim, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          523840    
                                                                 
 bidirectional (Bidirection  (None, None, 128)         66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 635329 (2.42 MB)
Trainable params: 635329 

In [8]:
# Set the training parameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model

In [10]:
NUM_EPOCHS = 10

# Train the model
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10


2024-03-06 16:56:32.602436: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 222298112 exceeds 10% of free system memory.
2024-03-06 16:56:32.973214: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 222298112 exceeds 10% of free system memory.
2024-03-06 16:56:32.973371: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 222298112 exceeds 10% of free system memory.
2024-03-06 16:56:33.137011: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 222298112 exceeds 10% of free system memory.
2024-03-06 16:56:44.067733: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 222298112 exceeds 10% of free system memory.


 2/98 [..............................] - ETA: 35:32 - loss: 0.6938 - accuracy: 0.5020  

: 

In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

# Plot the accuracy and results 
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")